In [278]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


In [279]:
torch.manual_seed(42)

In [280]:
device = torch.device("mps"  if torch.backends.mps.is_available() else "cpu")
print("Using Device: ", device )

Using Device:  mps


In [281]:
# df = pd.read_csv("fmnist_small.csv")
df = pd.read_csv("fashion-mnist_train.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [282]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [283]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [284]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [285]:
x_train = X_train/255.0
y_train = y_train/255.0

In [286]:
class CustomDataset(Dataset):

    def __init__(self, features, labels):
        
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [287]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [288]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

# Define Convoluation Neural Network

In [289]:
# class MyNN(nn.Module):
#     def __init__(self, input_features):
#         super().__init__()

#         self.features = nn.Sequential(
#             nn.Conv2d(input_features, 32, kernel_size=3, padding='same' ), # define the input channel in your image, out channel = filters, kernal_size = 3*3, padding= same padding
#             nn.ReLU(), 
#             nn.BatchNorm2d(32),
#             nn.MaxPool2d(kernel_size=2, stride=2),

#             # 2nd convolutional layer
#             nn.Conv2d( 32, 64, kernel_size=3, padding='same'), # 32 input features - becuase previous Conv layer ke filter next layer ke liye input ban jate hai
#             nn.ReLU(),
#             nn.BatchNorm2d(64),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#         )
#         self.classifier = nn.Sequential(
#             # Flatten
#             nn.Flatten(),
#             nn.Linear(64*7*7, 128),
#             nn.ReLU(),
#             nn.Dropout(p=0.4),

#             nn.Linear(128,64),
#             nn.ReLU(),
#             nn.Dropout(p=0.4),

#             nn.Linear(64, 10)

#         )

#     def forward(self, x):
#         x = self.features(x)
#         x = self.classifier(x)

#         return x

In [290]:
class MyNN(nn.Module):
    def __init__(self, input_features):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)

        return x

In [291]:
learning_rate = 0.01
epochs = 100

In [292]:
model = MyNN(1)  # 1 = number of input channels because it is white and black image
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [ ]:
# Training Loop
for epoch in range(epochs):
    
    total_epoch_loss = 0
    for batch_features, batch_labels in train_loader:
        #move data to gpu
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        # Forward pass
        outputs = model(batch_features)

        # calculate loss
        loss = criterion(outputs, batch_labels)

        # Back pss
        optimizer.zero_grad()
        loss.backward()

        # update the parameters
        optimizer.step()

        total_epoch_loss = total_epoch_loss + loss.item()

    avg_loss = total_epoch_loss/len(train_loader)
    print(f'Epoch: {epoch+1} , Loss: {avg_loss}')


/Users/abroadhub/Library/Python/3.9/lib/python/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch: 1 , Loss: 0.010237857916618092
Epoch: 2 , Loss: 0.0002170555681120921
Epoch: 3 , Loss: 0.00011648655397932354
Epoch: 4 , Loss: 7.677241206191591e-05
Epoch: 5 , Loss: 7.466092185428863e-05
Epoch: 6 , Loss: 6.975810794768774e-05
Epoch: 7 , Loss: 4.038705756257969e-05
Epoch: 8 , Loss: 4.6885978150322135e-05
Epoch: 9 , Loss: 3.569599502655304e-05
Epoch: 10 , Loss: 2.7403860548924927e-05
Epoch: 11 , Loss: 2.912151610124969e-05
Epoch: 12 , Loss: 3.3227780421164106e-05
Epoch: 13 , Loss: 2.550977305621682e-05
Epoch: 14 , Loss: 2.8567373054189993e-05
Epoch: 15 , Loss: 2.1954662316128194e-05
Epoch: 16 , Loss: 1.55812492390018e-05
Epoch: 17 , Loss: 2.213880139874469e-05
Epoch: 18 , Loss: 2.5368309322825412e-05
Epoch: 19 , Loss: 1.4529651030182815e-05
Epoch: 20 , Loss: 1.4432653676710908e-05
Epoch: 21 , Loss: 1.7725409475357908e-05
Epoch: 22 , Loss: 1.19381712543117e-05


In [ ]:
model.eval()

MyNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
# evaluate the TEST data
total = 0
correct = 0
with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.10266666666666667


In [ ]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

1.0
